In [0]:
from copy import deepcopy
import math
import random

In [0]:
class city:
	name = "_";
	x = 0
	y = 0

class stateClass:
	g = 0
	h = 0
	goal = False
	def __init__(self,cities,paths):
		self.cities = cities
		self.path = paths

In [0]:
def distance(city1, city2):
    return math.sqrt((city1.x - city2.x)**2 + (city1.y - city2.y)**2)

def printPath(path):
    for city in path:
        print(city.name)


def newCopyOfState(state):
    newState = stateClass
    newState.g = state.g
    newState.h = state.h
    newState.cities = deepcopy(state.cities)
    newState.path = deepcopy(state.path)
    return newState

def solve(cities):

    #keeps track of the number of states
    numNodes = 0

    path = []

    #add the first city to the path
    path.append(cities[0])
    cities.remove(cities[0])

    initalState = stateClass(cities,path)
    initalState.g = 0

    stateList = []
    stateList.append(initalState)

    #stores the upper bound, which will also be the shortest path found so far
    upperCost = 999999
    upperState = None


    def sort(state):
        #sorting by the huristic slowed things down fractionally.
        #sorting implies that we will find lower bounds faster, so that we can ignore more branches
        return state.g #+ state.h

    #the recursive function
    def compute(state):
        nonlocal stateList
        #global start
        nonlocal upperCost
        nonlocal upperState
        nonlocal numNodes

        #print(stateList)
        #print(upperCost)


        if state.g > upperCost:
            #This will usually not be reached, because we prune these out before they are made
            return

        #add to tge number of states
        numNodes += 1

        if len(state.cities) < 1:
            #a new upper bound is found
            if state.g < upperCost:
                upperCost = state.g
                upperState = state
            return

        #We no longer check for timeout
        #check for timeout
        #current = time.time()
        #if current - start >=300:
        #   print("timeout... current length: " + str(upperCost))
        #   printPath(upperState.path)
        #   sys.exit()

        stateList = []

        #test all possible next cities
        for c in range(len(state.cities)):
            #although a state is temporarily created, it is really check feasibility, and may get destroyed later.
            #i wont count this as a new state (yet) because it is really just a temporary variable holder
            oldDist = distance(state.path[-1],state.path[0])
            newState = stateClass(state.cities[:],state.path[:])
            newState.g = state.g
            newState.h = 0
            newState.path.append(newState.cities[c])
            newState.cities.remove(newState.cities[c])
            #we remove the path from the last city to the first city (which created a cycle)
            fakeDist =  distance(newState.path[-1], newState.path[0])
            #recalculate the new cyclic length
            newDist = distance(newState.path[-2], newState.path[-1]) + fakeDist
            #this represents the cost of this path
            newState.g += newDist - oldDist

            #our heuristic.  its the distance to the farthest, un-connected city from both the last node and the first node
            #we know that this will always be less than or equal to the real distance because we have to connect that node somehow
            #the shortest possible way is if it was the last node left to connect and so we directly connect it to both te first and last node
            #to complete the cycle... and that distacne is exactly what the heuristic calculates
            maxDist = fakeDist
            to = None
            for r in newState.cities:
                d = distance(newState.path[-1],r) + distance(r,newState.path[0])
                if d > maxDist:
                    maxDist = d
                    to = r

            #This will always be positive
            newState.h = maxDist - fakeDist

            #if our huristic causes a cost over the upperBound, then we know that this course of action is perilious
            if upperCost >= newState.g + newState.h:
                stateList.append(newState)


        #sort by the current cost
        stateList.sort(key = sort)


        #recurse over all states
        for s in stateList:
            compute(s)

    #start recursion
    compute(initalState)

    return upperState.path



In [8]:
num_cities = 16
data_points = 1000

for i in range(data_points):

    cities = []

    for c in range(num_cities):
        new_city = city()
        new_city.name = c
        new_city.x = random.randint(0,100)
        new_city.y = random.randint(0,100)
        cities.append(new_city)

        print(f"{new_city.x},{new_city.y},", end='')

    solution = solve(cities)

    for p in solution:
        print(f"{p.name},", end='')

    print(f"{i}")

30,86,70,87,58,83,25,26,24,97,25,99,74,45,55,43,9,64,20,72,46,92,19,51,47,88,60,59,82,68,90,57,0,4,5,10,12,2,1,14,15,6,13,7,3,11,8,9,0
57,55,48,51,33,39,43,88,1,71,3,29,15,63,63,96,1,56,18,47,82,2,85,65,60,83,55,22,5,15,74,66,0,1,2,9,6,4,8,5,14,13,10,11,15,12,7,3,1
93,9,87,54,2,22,32,61,9,93,19,20,98,99,99,56,39,2,39,21,19,81,47,33,39,68,62,57,26,85,83,47,0,15,1,7,6,13,12,14,4,10,3,2,5,8,9,11,2
25,37,65,91,4,35,46,52,5,25,10,35,80,9,17,6,75,21,24,10,33,96,2,51,23,100,22,79,69,1,99,57,0,3,13,12,10,1,15,8,6,14,9,7,4,2,11,5,3
18,15,37,91,56,21,34,57,2,95,10,37,77,93,75,45,29,8,7,59,82,34,43,49,12,81,76,98,16,99,84,0,0,8,2,15,10,7,6,13,1,14,4,12,9,3,11,5,4
98,96,63,86,88,1,45,38,66,70,12,75,49,56,83,58,82,58,85,4,33,75,59,47,82,51,48,45,79,57,85,87,0,7,8,14,12,2,9,3,13,11,6,5,10,4,1,15,5
34,6,63,82,50,4,42,40,18,10,100,43,93,97,43,40,6,98,78,63,34,69,55,69,94,59,49,70,98,65,88,58,0,4,3,7,10,8,13,11,1,6,9,15,12,14,5,2,6
18,24,50,79,8,38,76,81,49,99,52,32,97,38,95,10,29,43,94,78,5,78,76,48,8

KeyboardInterrupt: ignored